# Weekly Rolling

In [ ]:
import pandas as pd

def brandEquityScore_weekly(file_name, output_filename):
    
    #Output dataframe
    df_score = pd.DataFrame()
    
    #Output excel file
    writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
    workbook = writer.book
    
    #Metrics label
    xls = pd.ExcelFile(file_name, on_demand = True)
    equity_metrics = xls.sheet_names[1:]
    
    #Gathering brand names
    label_row = pd.read_excel(file_name, sheet_name=2)
    brand_row = label_row.iloc[4,:]
    brands = []
    for j in brand_row:
        if j != 0:
            brands.append(j)
    brands = brands[1:]
    
    #looping through each tab
    for i, n in enumerate(equity_metrics):
        df = pd.read_excel(file_name, skiprows=6 ,sheet_name=i+1, header=[0],index_col='Unnamed: 0')
    
    #Gather all column names containing either 'Score' or 'Attention'
        col = []
        for k in df.columns:
            if 'Score' in k:
                col.append(k)
            elif 'Attention' in k:
                col.append(k)
                
        df_score = df.loc[:,col]
        df_score.columns = brands
    
    #Filter Sunday date records            
        df_score.index.rename('Week end date')       
        df_filter = df_score[df_score.index.dayofweek == 6]
    
    #Generate an excel file with filtered records
        df_output = df_filter.to_excel(writer, sheet_name=n)
        
    writer.save()
    workbook.close()

# Melt

In [ ]:
import pandas as pd

def brandEquityScore_melt(file_name, output_filename):
    
    #read the input file into pandas dataframe
    equity = pd.ExcelFile(file_name)
    
    #column headers of the melted output
    cols = ['brand', 'date', 'metric', 'score']
    #output dataframe with the columns created above
    df = pd.DataFrame(columns=cols)
    
    #collect brand names
    brands_df = pd.read_excel(equity, sheet_name=1)
    brands = brands_df.columns
    
    #loop through each tab
    for i, n in enumerate(equity.sheet_names):
        each = pd.read_excel(equity, sheet_name=i, index_col=None)
        
    #change the dataframe to non-indexed     
        each.reset_index(inplace=True)
    #change the column header 'index' to 'date'
        each.rename(columns={'index':'date'},inplace=True)
    #melt columns    
        each_melt = pd.melt(each, id_vars=['date'], value_vars=brands, var_name='brand', value_name='score')
    #name each sheet with metric labels    
        each_melt['metric'] = equity.sheet_names[i]
    #append melted columns into one dataframe
        df = df.append(each_melt)
    #filter columns from dataframe with appended columns above
        df = df[cols]
    
    #generate an output file in csv format
        df.to_csv(output_filename, encoding='utf-8', index = False)